In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('2024-04-30T15-00_20-00_MononElmStreetNB.csv', parse_dates=['capture_ts'], index_col='capture_ts', date_format='ISO8601').drop(columns=['detection_id'])

In [ ]:
# Use DataFrame.describe() to get a summary of the data (these do not make a ton of sense for this kind of data)
df.describe()

### Visualize traffic over time

In [ ]:
# Plot object count over time (count unique object_id per minute)
object_count = df['object_id'].resample('1min').nunique()
object_count.plot()


In [ ]:
# Smooth the object count data by applying a rolling mean (e.g. windows size of 10)
object_count.rolling(window=10).mean().plot()

In [ ]:
# Plot the same as above, but differentiate between the different class_ids (use different colors and correct labels; 0: person, 1: bicycle, 2: car)
CLASSNAME_BY_ID = {0: 'person', 1: 'bicycle', 2: 'car'}

for class_id in df['class_id'].unique():
    class_count = df[df['class_id'] == class_id]['object_id'].resample('1min').nunique()  
    class_count.rolling(window=10).mean().plot(label=CLASSNAME_BY_ID[class_id])
plt.ylabel('avg object count per minute')
plt.legend()


### Plot object centers

In [ ]:
CLASSNAME_BY_ID = {0: 'person', 1: 'bicycle', 2: 'car'}

# Calculate the center of each bounding box and add two new columns to the DataFrame: 'center_x' and 'center_y'
df['center_x'] = (df['min_x'] + df['max_x']) / 2
df['center_y'] = (df['min_y'] + df['max_y']) / 2

# Plot the center of each bounding box over time (use different colors and correct labels)
plt.figure(figsize=(16, 9))
for class_id in df['class_id'].unique():
    class_data = df[df['class_id'] == class_id]
    plt.scatter(class_data['center_x'], class_data['center_y'], label=CLASSNAME_BY_ID[class_id], s=2)
plt.xlabel('center_x')
plt.ylabel('center_y')
plt.legend(loc='upper right')

# Make this resemble the original camera image by setting the origin to the top left and scaling the axis to a 16:9 aspect ratio
plt.gca().invert_yaxis()
plt.gca().set_aspect(0.5625, adjustable='box')


In [ ]:
# Plot the same as above, but use bounding box size as a color gradient and plot each object class into its own subplot (with its own gradient scale and colorbar)
for class_id in df['class_id'].unique():
    class_data = df[df['class_id'] == class_id]
    plt.figure(figsize=(16, 9))
    plt.scatter(class_data['center_x'], class_data['center_y'], c=(class_data['max_x'] - class_data['min_x']) * (class_data['max_y'] - class_data['min_y']), label=CLASSNAME_BY_ID[class_id], s=2)
    plt.colorbar()
    plt.xlabel('center_x')
    plt.ylabel('center_y')
    plt.gca().invert_yaxis()
    plt.gca().set_aspect(0.5625, adjustable='box')
    plt.title(CLASSNAME_BY_ID[class_id])
    plt.show()
